In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

# Load CSV data from Part I into a dataframe
weather_df = pd.read_csv('output_data/cities.csv').drop(['Unnamed: 0'],axis=1)
weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,hithadhoo,MV,-0.6000,73.0833,82.31,64,10,4.83
1,carnarvon,AU,-24.8667,113.6333,62.60,67,0,5.75
2,yellowknife,CA,62.4560,-114.3525,9.90,52,40,13.80
3,cherskiy,RU,68.7500,161.3000,21.40,83,60,8.39
4,maceio,BR,-9.6658,-35.7353,77.00,88,0,6.91
...,...,...,...,...,...,...,...,...
562,puteyets,RU,65.1716,57.0910,37.69,91,100,4.29
563,paracuru,BR,-3.4100,-39.0306,82.00,75,40,13.00
564,ambunti,PG,-4.2365,142.8448,78.46,91,62,1.66
565,cotonou,BJ,6.3654,2.4183,86.00,79,20,10.36


In [57]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Pull data from main dataframe for heatmap layer
locations = weather_df[['Latitude','Longitude']].astype(float)
humidity_weight = weather_df['Humidity (%)']

figure_layout = {'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'}

fig=gmaps.figure(figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight,
                                 dissipating=False, max_intensity=100,
                                point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Create Dataframe fitting weather criteria
best_vacation = weather_df.loc[(weather_df['Temperature (F)'] <= 80) & (weather_df['Temperature (F)'] >= 70) & 
                               (weather_df['Wind Speed (mph)'] <= 10) & (weather_df['Cloudiness (%)'] == 0), :]

# Drop any rows with missing data
best_vacation.dropna(how='any')

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
4,maceio,BR,-9.6658,-35.7353,77.00,88,0,6.91
22,karratha,AU,-20.7377,116.8463,74.84,33,0,9.01
175,umm lajj,SA,25.0213,37.2685,77.52,73,0,6.17
200,bhadravati,IN,13.8667,75.7167,75.54,77,0,2.77
413,bandrele,YT,-12.9067,45.1914,78.80,89,0,2.30
491,oussouye,SN,12.4850,-16.5469,72.46,94,0,9.22
527,kavali,IN,14.9167,79.9833,77.02,88,0,8.72


In [32]:
# Hotel Map
hotel_df = weather_df.copy()
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,hithadhoo,MV,-0.6000,73.0833,82.31,64,10,4.83
1,carnarvon,AU,-24.8667,113.6333,62.60,67,0,5.75
2,yellowknife,CA,62.4560,-114.3525,9.90,52,40,13.80
3,cherskiy,RU,68.7500,161.3000,21.40,83,60,8.39
4,maceio,BR,-9.6658,-35.7353,77.00,88,0,6.91
...,...,...,...,...,...,...,...,...
562,puteyets,RU,65.1716,57.0910,37.69,91,100,4.29
563,paracuru,BR,-3.4100,-39.0306,82.00,75,40,13.00
564,ambunti,PG,-4.2365,142.8448,78.46,91,62,1.66
565,cotonou,BJ,6.3654,2.4183,86.00,79,20,10.36


In [34]:
# geocoordinates
hotel_name = []
hotel_address = []
lats = hotel_df['Latitude']
lngs = hotel_df['Longitude']
x=0

for city in hotel_df['City']:
    target_coordinates = f'{lats[x]},{lngs[x]}'
    target_radius = 5000
    target_type = "lodging"
    x += 1

    # Set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)

    temp = response.json()
    try:
        hotel_name.append(temp['results'][0]['name'])
        hotel_address.append(temp['results'][0]['vicinity'])
        print(f'Hotel in {city} found! Finding address...')
    except:
        hotel_name.append('N/A')
        hotel_address.append('N/A')
        print(f'No hotels in {city}.')
        
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Address'] = hotel_address

Hotel in hithadhoo found! Finding address...
Hotel in carnarvon found! Finding address...
Hotel in yellowknife found! Finding address...
Hotel in cherskiy found! Finding address...
Hotel in maceio found! Finding address...
No hotels in marovoay.
No hotels in tilichiki.
Hotel in hofn found! Finding address...
Hotel in marystown found! Finding address...
Hotel in barrow found! Finding address...
Hotel in busselton found! Finding address...
Hotel in georgetown found! Finding address...
No hotels in raga.
No hotels in albany.
Hotel in nyurba found! Finding address...
Hotel in portoferraio found! Finding address...
Hotel in puerto ayora found! Finding address...
Hotel in butaritari found! Finding address...
Hotel in beppu found! Finding address...
Hotel in wewak found! Finding address...
Hotel in cidreira found! Finding address...
Hotel in tasiilaq found! Finding address...
Hotel in karratha found! Finding address...
Hotel in olavarria found! Finding address...
Hotel in codrington found! Fi

Hotel in bira found! Finding address...
Hotel in fort nelson found! Finding address...
Hotel in bac lieu found! Finding address...
Hotel in maragogi found! Finding address...
Hotel in turinsk found! Finding address...
Hotel in bhadravati found! Finding address...
Hotel in ahuimanu found! Finding address...
Hotel in cabo san lucas found! Finding address...
Hotel in plastun found! Finding address...
Hotel in cockburn town found! Finding address...
Hotel in klaksvik found! Finding address...
Hotel in kahului found! Finding address...
Hotel in talcahuano found! Finding address...
No hotels in saskylakh.
Hotel in port elizabeth found! Finding address...
Hotel in nikolskoye found! Finding address...
Hotel in san patricio found! Finding address...
Hotel in tromso found! Finding address...
Hotel in sri aman found! Finding address...
Hotel in chadiza found! Finding address...
Hotel in kisangani found! Finding address...
Hotel in alpena found! Finding address...
Hotel in tamandare found! Finding

Hotel in sept-iles found! Finding address...
Hotel in morden found! Finding address...
Hotel in sechura found! Finding address...
Hotel in mercedes found! Finding address...
Hotel in laukaa found! Finding address...
Hotel in lorengau found! Finding address...
Hotel in hay river found! Finding address...
Hotel in tanete found! Finding address...
Hotel in buchanan found! Finding address...
Hotel in nara found! Finding address...
Hotel in purpe found! Finding address...
Hotel in maniitsoq found! Finding address...
Hotel in algiers found! Finding address...
Hotel in maryborough found! Finding address...
Hotel in manaure found! Finding address...
Hotel in rijssen found! Finding address...
Hotel in cap malheureux found! Finding address...
Hotel in galeana found! Finding address...
Hotel in helena found! Finding address...
Hotel in nanortalik found! Finding address...
Hotel in te anau found! Finding address...
Hotel in general pico found! Finding address...
Hotel in palembang found! Finding a

In [36]:
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name,Hotel Address
0,hithadhoo,MV,-0.6000,73.0833,82.31,64,10,4.83,Scoop Guest House,"Scoop Guest House, S. Hithadhoo - Rep Of Maldi..."
1,carnarvon,AU,-24.8667,113.6333,62.60,67,0,5.75,Hospitality Carnarvon,"6 West Street, South Carnarvon"
2,yellowknife,CA,62.4560,-114.3525,9.90,52,40,13.80,The Explorer Hotel,"4825 49th Avenue, Yellowknife"
3,cherskiy,RU,68.7500,161.3000,21.40,83,60,8.39,Gostinitsa,"ул.Пушкина, 34 кв.9, Chersky"
4,maceio,BR,-9.6658,-35.7353,77.00,88,0,6.91,Poutur Pousada,"Parque Rodolfo Lins, 284 - Centro, Maceió"
...,...,...,...,...,...,...,...,...,...,...
562,puteyets,RU,65.1716,57.0910,37.69,91,100,4.29,"Sot ""Agrarnik""","Unnamed Road, Puteets"
563,paracuru,BR,-3.4100,-39.0306,82.00,75,40,13.00,Hotel Vento Brasil,"R. Ormezinda Sampaio, 240 - Centro, Paracuru"
564,ambunti,PG,-4.2365,142.8448,78.46,91,62,1.66,Ambunti Lodge,"Riviere Sepik, Wewak 531"
565,cotonou,BJ,6.3654,2.4183,86.00,79,20,10.36,Novotel Cotonou Orisha,"1821 Boulevard de la Marina, Cotonou"


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [58]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))